In [ ]:
import shutil
import random
import glob
import os
names = ["A","B","C"]
os.makedirs("./test", exist_ok=True)

for name in names:
    in_dir = "./face/"+name+"/*"
    in_jpg=glob.glob(in_dir)
    img_file_name_list=os.listdir("./face/"+name+"/")
    random.shuffle(in_jpg)
    os.makedirs('./test/' + name, exist_ok=True)
    for t in range(len(in_jpg)//5):
        shutil.move(str(in_jpg[t]), "./test/"+name)

In [ ]:
import os
import cv2
import glob
from scipy import ndimage
"""
faceディレクトリから画像を読み込んで回転、ぼかし、閾値処理をしてtrainディレクトリに保存する.
"""
names = ["A","B","C"]
os.makedirs("./train", exist_ok=True)
for name in names:
    in_dir = "./face/"+name+"/*"
    out_dir = "./train/"+name
    os.makedirs(out_dir, exist_ok=True)
    in_jpg=glob.glob(in_dir)
    img_file_name_list=os.listdir("./face/"+name+"/")
    for i in range(len(in_jpg)):
        img = cv2.imread(str(in_jpg[i]))
        for ang in [-10,0,10]:
            img_rot = ndimage.rotate(img,ang)
            img_rot = cv2.resize(img_rot,(64,64))
            fileName=os.path.join(out_dir,str(i)+"_"+str(ang)+".jpg")
            cv2.imwrite(str(fileName),img_rot)
            # 閾値
            img_thr = cv2.threshold(img_rot, 100, 255, cv2.THRESH_TOZERO)[1]
            fileName=os.path.join(out_dir,str(i)+"_"+str(ang)+"thr.jpg")
            cv2.imwrite(str(fileName),img_thr)
            # ぼかし
            img_filter = cv2.GaussianBlur(img_rot, (5, 5), 0)
            fileName=os.path.join(out_dir,str(i)+"_"+str(ang)+"filter.jpg")
            cv2.imwrite(str(fileName),img_filter)

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

name = ["A","B","C"]

# 教師データのラベル付け
X_train = []
Y_train = []
for i in range(len(name)):
    img_file_name_list=os.listdir("./train/"+name[i])
    print(len(img_file_name_list))
    for j in range(0,len(img_file_name_list)-1):
        n=os.path.join("./train/"+name[i]+"/",img_file_name_list[j])
        img = cv2.imread(n)
        b,g,r = cv2.split(img)
        img = cv2.merge([r,g,b])
        X_train.append(img)
        Y_train.append(i)

# テストデータのラベル付け
X_test = [] # 画像データ読み込み
Y_test = [] # ラベル（名前）
for i in range(len(name)):
    img_file_name_list=os.listdir("./test/"+name[i])
    print(len(img_file_name_list))
    for j in range(0,len(img_file_name_list)-1):
        n=os.path.join("./test/"+name[i]+"/",img_file_name_list[j])
        img = cv2.imread(n)
        b,g,r = cv2.split(img)
        img = cv2.merge([r,g,b])
        X_test.append(img)
        # ラベルは整数値
        Y_test.append(i)
X_train=np.array(X_train)
X_test=np.array(X_test)

from keras.layers import Activation, Conv2D, Dense, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.utils.np_utils import to_categorical

y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

# モデルの定義
model = Sequential()
model.add(Conv2D(input_shape=(64, 64, 3), filters=32,kernel_size=(3, 3),
                 strides=(1, 1), padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3, 3),
                 strides=(1, 1), padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3, 3),
                 strides=(1, 1), padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256))
model.add(Activation("sigmoid"))
model.add(Dense(128))
model.add(Activation('sigmoid'))
model.add(Dense(3))
model.add(Activation('softmax'))

# コンパイル
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 学習
history = model.fit(X_train, y_train, batch_size=32,
                    epochs=50, verbose=1, validation_data=(X_test, y_test))

# 汎化制度の評価・表示
score = model.evaluate(X_test, y_test, batch_size=32, verbose=0)
print('validation loss:{0[0]}\nvalidation accuracy:{0[1]}'.format(score))

#acc, val_accのプロット
plt.plot(history.history["acc"], label="acc", ls="-", marker="o")
plt.plot(history.history["val_acc"], label="val_acc", ls="-", marker="x")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
plt.show()

#モデルを保存
model.save("my_model.h5")

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import cv2, time
from datetime import datetime
import slackweb
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import  load_model
import sys

#proxy認証
#os.environ["HTTP_PROXY"] = "proxy_adress"
#os.environ["HTTPS_PROXY"] = "proxy_adress"

# 正面の顔検出用
cascPath = '-----Your_Path-----/lbpcascades/lbpcascade_frontalface.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

model = load_model('./my_model.h5')

#Slackに通知を飛ばす
#slack = slackweb.Slack(url="<https://hooks.slack.com/services/-----Your_adress-----")

def detect_who(img):
    #予測
    name=""
    print(model.predict(img))
    nameNumLabel=np.argmax(model.predict(img))
    if nameNumLabel== 0: 
        name="A"
    elif nameNumLabel==1:
        name="B"
    elif nameNumLabel==2:
        name="C"
    return name

#MJPEGStream
URL = "-----raspberry_address------/?action=stream"
s_video = cv2.VideoCapture(URL)

while True:
  ret, img = s_video.read()

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray = cv2.equalizeHist( gray )
  faces = faceCascade.detectMultiScale(gray, 1.1, 3, 0, (10, 10))

  for (x, y, w, h) in faces:
      # 見つかった顔を矩形で囲む
      cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
      time = datetime.now().strftime('%Y%m%d%H%M%S.%f')[:-3]
      path = '.\\\\img\\\\' + time + '.jpg'
      cut_face = img[y:y+h,x:x+w,:]
      cut_face_resize = cv2.resize(cut_face,(64,64))
      cv2.imwrite(path, cut_face_resize)
      cut_face_resize=np.expand_dims(cut_face_resize,axis=0)
      name = detect_who(cut_face_resize)
      cv2.putText(cut_face_resize, name, (x, y+h+20), cv2.FONT_HERSHEY_DUPLEX, 1, (255,0,0), 2)
      if name == "A":
        # slack.notify(text=" "+name)
    
  cv2.imshow("Face-Detect-Video",img)
  key = cv2.waitKey(1) & 0xff
  if key == ord('q'): break